In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import torch.backends.cudnn as cudnn
from datasets.utils import build_dataset, get_ood_dataloaders
from models.utils import build_model
from utils.eval import *
from utils.loss import LogitNormLoss
from utils.posthoc import get_optimal_temperature


seed = 1
num_to_avg = 10
gpu = 0
model_type = 'wrn'
loss_function = 'normal'
dataset =  'cifar10'
method_name = '_'.join([dataset, model_type, loss_function, 'standard'])
print("method_name: " + method_name)

ood_score = 'MSP'
test_bs = 200
num_classes = 10
input_size = 32
input_channels = 3
mean=(0.492, 0.482, 0.446)
std=(0.247, 0.244, 0.262)

prefetch_threads = 4
save_path = './snapshots/'


method_name: cifar10_wrn_normal_standard


In [2]:
torch.manual_seed(seed)
np.random.seed(seed)
cudnn.benchmark = True  # fire on all cylinders

if gpu is not None:
    device = torch.device('cuda:{}'.format(int(gpu)))
    torch.cuda.manual_seed(seed)
else:
    device = torch.device('cpu')

# Create model and load weights from .pt file
net = build_model(model_type, num_classes, device, load=True, path=save_path, filename=method_name)

net.eval()
cudnn.benchmark = True  # fire on all cylinders

# load ID test dataset
test_data = build_dataset(dataset, mode="test", size=input_size, channels=input_channels,
                          mean=mean, std=std)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=test_bs, shuffle=False,
                                          num_workers=prefetch_threads, pin_memory=True)

# optimize the temperature scale T with the test dataset
optimal_t = get_optimal_temperature(net, test_loader, device)


Model restored! Epoch: 199
Files already downloaded and verified
Before temperature - NLL: 0.205, ECE: 0.031
Optimal temperature: 1.541
After temperature - NLL: 0.168, ECE: 0.010


In [4]:
# /////////////// OOD Detection ///////////////
ood_num_examples = len(test_data) // 5
OOD_data_list = ["MNIST", "GTSRB", "Textures", "SVHN", "Places365", "LSUN-C", "LSUN-R", "cifar10"] #  "Gaussian"

ood_loader_dict = get_ood_dataloaders(OOD_data_list, input_size=input_size, input_channels=input_channels, 
                                      mean=mean, std=std, test_bs=test_bs, prefetch_threads=prefetch_threads)


get_all_measures(net, test_loader, ood_loader_dict, ood_num_examples, device, optimal_t, ood_score, 
                     recall_level=0.95, test_bs=test_bs, to_string=True, method_name=method_name)

Files already downloaded and verified
Error Rate 5.26
				cifar10_wrn_normal_standard
FPR95:			34.22
AUROC: 			92.85
AUPR:  			99.50


ECE Error
ECE Error 1.00


MNIST Detection
